# Poisson 1D

This notebook solves the linear problem:

$$ -u(x)'' = f(x) \text{ in } \Omega = [-1,1]$$ 

with

$$ f(x) = \frac{\pi^2}{4} \cos{\left(\frac{\pi x}{2}\right)} $$

and homogeneous Dirichlet boundary conditions.

In [ ]:
%matplotlib inline
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl

# Generate code for evaluate_basis_derivatives for testing
parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

u_exact = lambda x: np.cos(np.pi*x/2)

def poisson1d(V):
    deg = V.ufl_element().degree()
    def boundary(x):
        return x[0] < -1 + DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4", degree=deg)

    a = u.dx(0)*v.dx(0)*dx
    L = f*v*dx

    u = Function(V)
    solve(a == L, u, bc)
    return u

In [ ]:
#mesh = UnitIntervalMesh(2)
#MeshTransformation.translate(mesh, Point(1.0))
mesh = IntervalMesh(2, -1, 1)

In [ ]:
V = FunctionSpace(mesh, "Lagrange", 3)
u = poisson1d(V)

In [ ]:
xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0,1.1))
_ = pl.legend()

# Using Hermite elements

Fail: first derivatives are set to 0.

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d(V)

xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0, 1.1))
_ = pl.legend()

# Manually rebuilding the mass matrix

Hermite dofs are set to zero by bc.apply(): we manually find those and reset the relevant rows of the mass matrix.

**NOTE** that we set the linear algebra backend to Eigen because the default PETSc uses sparse matrices whose sparsity patterns are initialised / fixed by assemble(), so we cannot freely change entries unless we directly talk to PETSc.

This can be done with petsc4py (which is available in the docker container), see [this demo](https://bitbucket.org/cwilson/dolfin/src/883fcc4a12d53c7dae1bb9d5fc7e7302051b0bae/demo/undocumented/petsc4py/python/demo_petsc4py.py?at=master&fileviewer=file-view-default), but I'm too lazy now.

In [ ]:
parameters['linear_algebra_backend'] = 'Eigen'
mesh = IntervalMesh(2, -1, 1)
V = FunctionSpace(mesh, "Hermite", 3)
deg = V.ufl_element().degree()

def boundary(x):
    return x[0] < -1 + DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

# Need to project to work around lack of evaluate_dofs() for Hermite
u0 = project(Constant(0.0), V)
bc = DirichletBC(V, u0, boundary)

u = TrialFunction(V)
v = TestFunction(V)
f = Expression("pi*pi*cos(pi*x[0]/2)/4", degree=deg)

a = u.dx(0)*v.dx(0)*dx
L = f*v*dx

**Plan:** Mimic some of the steps in `dolfin/fem/DirichletBC.cpp, compute_bc_pointwise()` to find out which are the dofs at boundary points marked by the Dirichlet boundary condition. Then manually assemble the system, apply the BCs and undo the changes to the rows corresponding to Hermite elements.

**Update:** It's easier to use:

In [ ]:
bc.get_boundary_values().keys()

But in this very simple example we can deduce that from:

In [ ]:
element = V.element()
dofmap = V.dofmap()

dofmap.cell_dofs(0), dofmap.cell_dofs(1)

Based on the usual ordering, dofs 3 and 5 should be the Hermite ones. So we need to copy back those two rows into the mass matrix and the RHS.

In [ ]:
A = assemble(a)
A0 = A.copy()  # Will need this to restore rows later
b = assemble(L)
b0 = b.copy()  # ditto

bc.apply(A, b) # Applying BCs messes things up for dofs 3, 5

In [ ]:
b[[3,5]] = b0[[3,5]]

There is a [bug](https://bugs.launchpad.net/dolfin/+bug/1063868) in the automatically generated documentation which affects set() below. Only one function with three parameters is actually exposed by SWIG and the second and last are actually the indices of the rows and columns to modify.

In [ ]:
block = A0.array()[[3,5]]

rows = np.array([3,5], dtype=np.intc)
cols = np.arange(6, dtype=np.intc)

A.set(block, rows, cols)
A.apply("insert")

In [ ]:
u = Function(V)
U = u.vector()
solve(A, U, b)

In [ ]:
xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0, 1.1))
_ = pl.legend()

YESSSSSSSSSS!

# Fixing DirichletBC



It seems like I need to define a special dofmap for Hermite elements which in dofmap.cell_dofs() returns only the Lagrange dofs (?). Won't this break lots of other things? Instead I could try to construct a copy of the function space with the right dofmap for the BCs and pass it to DirichletBC's constructor.

## Another approach

Identify all the global dofs corresponding to Hermite functionals:

```
# Pseudocode:
A = assemble(a)
A0 = A.copy()
b = assemble(L)
b0 = F.copy()

for cell in cells:
   local_dofs = cell.dofs()
   hermite_dofs = filter(partial(is_hermite, element), local_dofs)
   global_hermite_dofs = dofmap.local_to_global(hermite_dofs)
   # Is it OK to rewrite all hermite dofs? Shouldn't I just use those in DirichletBC?
   copy_rows(global_hermite_dofs, from=A0, to=A)
   copy_rows(global_hermite_dofs, from=b0, to=b)
```

In [ ]:
#for cell in (Cell(mesh, i) for i in range(mesh.num_cells)):
for i, cell in (Cell(mesh, j) for j in range(mesh.num_cells())):
    hermite_dof_indices = [1,3]
    global_dofs = dofmap.cell_dofs(i)[hermite_dof_indices]
    print(global_dofs)

## Some tests

In [ ]:
cell = Cell(mesh, 0)
dofmap.dofs()

In [ ]:
dofmap.cell_dofs(0)

In [ ]:
bc.get_boundary_values()